In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
housing_data = pd.read_csv("Data\housing.csv")
census_data = pd.read_csv("Clean Data\census.csv")
housing_data.head(5)

,State Code,State Name,District Code,District Name,Tehsil Code,Tehsil Name,Town Code/Village code,Ward No,Area Name,Rural/Urban,...,Contition_T_Total,Contition_T_Good,Contition_T_Livable,Contition_T_Dilapidated,Contition_R_Total,Residence_Good,Residence_Livable,Residence_Dilapidated,Residence_cum_other,Residence_cum_Good
0,1,JAMMU & KASHMIR,1,Kupwara,0,Kupwara,0,0,District - Kupwara,Rural,...,100,33.6,58.1,8.3,90.9,32.7,51.0,7.2,9.1,1.0
1,1,JAMMU & KASHMIR,1,Kupwara,0,Kupwara,0,0,District - Kupwara,Total,...,100,34.8,57.3,7.9,91.0,33.8,50.2,6.9,9.0,0.9
2,1,JAMMU & KASHMIR,1,Kupwara,0,Kupwara,0,0,District - Kupwara,Urban,...,100,45.7,49.5,4.8,91.8,45.0,42.2,4.5,8.2,0.7
3,1,JAMMU & KASHMIR,2,Badgam,0,Badgam,0,0,District - Badgam,Rural,...,100,49.7,46.3,3.9,97.2,49.0,44.5,3.7,2.8,0.7
4,1,JAMMU & KASHMIR,2,Badgam,0,Badgam,0,0,District - Badgam,Total,...,100,51.6,44.7,3.8,97.3,50.8,43.0,3.5,2.7,0.8


In [9]:
columns_to_keep = ["District Name", "Rural/Urban", "Total Number of households", "Total Number of Livable", "Total Number of Dilapidated", "Latrine_premise"]
housing_data = housing_data[columns_to_keep]
housing_data.head(5)

,District Name,Rural/Urban,Total Number of households,Total Number of Livable,Total Number of Dilapidated,Latrine_premise
0,Kupwara,Rural,100,58.1,8.3,48.9
1,Kupwara,Total,100,57.3,7.9,50.9
2,Kupwara,Urban,100,49.5,4.8,70.2
3,Badgam,Rural,100,46.3,3.9,83.3
4,Badgam,Total,100,44.7,3.8,84.3


In [10]:
merged_data = pd.merge(housing_data, census_data[['District Name', 'Households', "Households_Rural", "Households_Urban"]], on='District Name', how='left')
print(merged_data.columns)


Index(['District Name', 'Rural/Urban', 'Total Number of households',
       'Total Number of Livable', 'Total Number of Dilapidated',
       'Latrine_premise', 'Households', 'Households_Rural',
       'Households_Urban'],
      dtype='object')


In [11]:
merged_data['Total Number of households'] = pd.to_numeric(merged_data['Total Number of households'], errors='coerce')
merged_data['Total Number of Livable'] = pd.to_numeric(merged_data['Total Number of Livable'], errors='coerce')
merged_data['Total Number of Dilapidated'] = pd.to_numeric(merged_data['Total Number of Dilapidated'], errors='coerce')
merged_data['Latrine_premise'] = pd.to_numeric(merged_data['Latrine_premise'], errors='coerce')
merged_data['Rural/Urban'] = pd.to_numeric(merged_data['Rural/Urban'], errors='coerce')

merged_data['Households_Rural'] = merged_data['Rural/Urban'] * merged_data['Total Number of households'] / 100
merged_data['Households_Urban'] = (100 - merged_data['Rural/Urban']) * merged_data['Total Number of households'] / 100
merged_data['Households_Rural_Livable'] = merged_data['Households_Rural'] * merged_data['Total Number of Livable'] / 100
merged_data['Households_Urban_Livable'] = merged_data['Households_Urban'] * merged_data['Total Number of Livable'] / 100
merged_data['Households_Rural_Dilapidated'] = merged_data['Households_Rural'] * merged_data['Total Number of Dilapidated'] / 100
merged_data['Households_Urban_Dilapidated'] = merged_data['Households_Urban'] * merged_data['Total Number of Dilapidated'] / 100
merged_data['Households_Rural_Toilet_Premise'] = merged_data['Households_Rural'] * merged_data['Latrine_premise'] / 100
merged_data['Households_Urban_Toilet_Premise'] = merged_data['Households_Urban'] * merged_data['Latrine_premise'] / 100

merged_data.rename(columns={'District Name': 'District'}, inplace=True)

merged_data.to_csv('Data\housing.csv', index=False)

housing_districts = set(housing_data['District Name'])
census_districts = set(census_data['District Name'])

missing_in_housing = census_districts - housing_districts
missing_in_census = housing_districts - census_districts

print("Districts missing in housing data:", missing_in_housing)
print("Districts missing in census data:", missing_in_census)

Districts missing in housing data: {'PONDICHERRY', 'North  AND Middle Andaman', 'Lakshadweep', 'Lahul AND Spiti', 'Dadra AND Nagar Haveli'}
Districts missing in census data: {'LAKSHADWEEP', 'Lahul & Spiti', 'Dadra & Nagar Haveli', 'Puducherry', 'North  & Middle Andaman'}


In [13]:
merged_data.to_csv("Clean Data/housing.csv", index=False)